# Pytorch tutorial


## What is pytorch?
- Numpy를 대체하며 GPU연산을 지원하는 tensor 구조 지원  
CUDA(NDVIA에서 gpu를 통한 연산을 위해 만든 API)와 cuDNN(CUDA를 활용하여 딥러닝 gpu연산을 지원하는 API)을 활용하여 연산 가속  
일반적으로 CUDA와 cuDNN을 활용하는 연산은 CPU연산의 15배정도 가속을 가져옴. 


- backward() 함수를 통한 그래프 미분 연산 지원

## Pytorch vs Tensorflow

#### 둘 모두 GPU를 활용하는 framework
- Tensorflow  
`Define and Run`방식  
연산 그래프를 미리 만들어두고, 실제 연산시 값을 전달하여 결과를 얻음(정적)  
직관적이지 않고 그래프를 정의하는 부분과 실행하는 부분이 분리되어 코드가 길어짐  
최적화에서 장점


- Pytorch  
`Define by Run` 방식  
연산 그래프를 미리 만들어두지 않고 값이 할당되어 전달되는 과정에서 그래프가 작성(동적)  
직관적이고 간단한 코드  
(Pytorch측의 주장으로는)Tensorflow보다 평균 2.5배정도 빠른 속도(적어도 밀리지는 않음)

## Tensor

Tensor: Numpy의 ndarray와 유사한 matrix자료구조. GPU연산 가속이 가능

In [ ]:
import torch
import numpy as np

### Tensor 생성

In [ ]:
not_initialized = torch.empty(3, 4)
not_initialized

In [ ]:
random_initialized = torch.rand(3, 4)
random_initialized

In [ ]:
zero_initialized = torch.zeros(3, 4)
zero_initialized

In [ ]:
list_initialized = torch.tensor([[1, 2],[3, 4]])
list_initialized

In [ ]:
arange_lnitiaized = torch.arange(10)
arange_lnitiaized

### Tensor와 numpy간의 변환

In [ ]:
x = torch.ones(10)
x, type(x)

In [ ]:
y = x.numpy()
y, type(y)

In [ ]:
z = torch.from_numpy(y)
z

#### 주의사항 - tensor와 ndarray는 메모리공간을 공유함

In [ ]:
x.add_(1)

In [ ]:
x

In [ ]:
y, z

### Tensor 특성

In [ ]:
list_initialized.shape, list_initialized.ndim

In [ ]:

list_initialized.size()

### Tensor reshape vs view

-  두 함수는 기본적으로 같은 기능을 하나, view는 shallow copy이다.
-  그러나, reshape이 deep copy인 것은 아니다. reshape은 상황에 따라 deep/shallow를 넘나든다.
-  Reshape은 Contiguous한 operate를 지원하나, view는 지원하지 않는다.

In [ ]:
list_initialized.t()

In [ ]:
list_initialized.t().reshape(-1, 1)

In [ ]:
list_initialized.t().view(-1, 1) # Error!

### Tensor의 배열 연산

인덱싱, 슬라이싱, 선형대수 등 각종 연산 numpy와 동일하게 수행 가능

#### 기본연산

In [ ]:
x = torch.arange(1, 11).reshape(2, 5)
y = torch.arange(1, 20, step=2).reshape(2, 5)
x, y

In [ ]:
x + y, torch.add(x, y)

In [ ]:
x - y, torch.sub(x, y)

In [ ]:
x * y

In [ ]:
y % x

#### 행렬곱

In [ ]:
x.matmul(y.reshape(5, 2))

#### 바꿔치기(in-place) 와 반환하기(out-of-place)

In [ ]:
x

In [ ]:
x.add(3)

In [ ]:
x

In [ ]:
x.add_(3)

In [ ]:
x

#### 헷갈리는 경우

In [ ]:
z = x.numpy()
x = x.add(3)
print('x:',x)
print('z:',z)
print('========================')
x.add_(3)
print('x:', x)
print('z:', z)

#### Indexing and slicing

In [ ]:
x[0, 3]

In [ ]:
x[:, 1:4]

### Aggregation

In [ ]:
x

In [ ]:
x.sum()

In [ ]:
x.sum(dim=0), x.sum(axis=0)

In [ ]:
x.sum(dim=1), x.sum(axis=1)

#### Can't use mean() on integers

In [ ]:
x.dtype, x

In [ ]:
try:
    print(x.mean(dim=0))
except Exception as exc:
    print(exc)

In [ ]:
# x = torch.arange(1, 11, dtype=float).reshape(2, 5)
x = x.float()
x

In [ ]:
x.mean(), x.mean(dim=0)

### Broadcasting

In [ ]:
x = torch.arange(1, 11).reshape(1, -1)
y = torch.arange(1, 20, step=2).reshape(-1, 1)

In [ ]:
x + y

## AUTOGRAD: 자동미분

`pytorch` 신경망의 중심. `backward()`를 가능케 하는 패키지  
`Tensor`의 모든 연산에 대해 자동 미분을 제공

`pytorch.Tensor` class에는 `.required_grad` 속성이 존재  
`.required_grad`를 True로 세팅하면 해당 tensor를 기반으로 이루어신 모든 연산을 추적(track)하기 시작함  
모든 연산이 완료된 후, `backward()`를 호출하면 모든 gradient가 자동으로 계산(`Tensor.grad` 속성에 gradient가 누적됨)

Autograd의 다른 중요 class는 `Function`class  
`Tensor`와 `Function`class는 연결되어 있으며, 모든 연산을 encode하여 acyclic graph를 생성  
각각의 tensor는 `.grad_fn`속성을 가지며 이는 `Tensor`를 생성한 `Function`을 참조함

In [ ]:
x = torch.ones(5, 2, requires_grad=True)
# x = torch.arange(10, dtype=torch.float32).reshape(5, 2)
# x.requires_grad = True
x

In [ ]:
y = x + 10

In [ ]:
y

y는 연산의 결과로 생성된 tensor이므로 grad_fn을 가짐

In [ ]:
y.grad_fn

In [ ]:
z = y * y * 10

In [ ]:
out = z.mean()

In [ ]:
z, out

### 변화도(Gradient) 계산)

In [ ]:
out.backward(retain_graph=True)

In [ ]:
print(x.grad)

#### Jacobian matrix 활용

In [ ]:
x = torch.ones(5, 2, requires_grad=True)
y = x + 10
z = y * y * 10
out = z.mean(axis=1)
out

In [ ]:
x = torch.ones(5, 2, requires_grad=True)
y = x + 10
y = y * y * 10
y = y.mean(axis=1)
y

In [ ]:
y.backward(torch.tensor([1., 3.,1.,1.,1.]))

In [ ]:
print(x.grad)

## DataLoader

torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,  num_workers=0,...)

데이터를 학습모델에 통째로 넣지 않고 batch 단위로 나누어 학습시킬 때 활용.  
모든 데이터를 나누고 나눈 데이터를 차례로 넣어주는 과정을 대신해줌
- 용어  
 - epoch: 모든 training data 혹은 모든 training data를 학습시킨 상태
 - batch: batch_size만큼의 training data만 학습시키기 위한 소분 데이터


In [ ]:
x = [[73, 80, 75],
   [93, 88, 93],
   [89, 91, 90],
   [96, 98, 100],
   [73, 66, 70]]
x = torch.FloatTensor(x)
x

In [ ]:
data_loader = torch.utils.data.DataLoader(x,
                            batch_size=2, 
                            shuffle=True, 
                            num_workers=0)

In [ ]:
epochs = 2
for epoch in range(epochs):
    for step, batch in enumerate(data_loader):
        print(step, batch)

## CUDA Tensor

### GPU로 이동

In [ ]:
if torch.cuda.is_available():
    x = x.to(torch.device('cuda'))
    print(type(x))
    print(x.device)

#### 생성과 함께 gpu에 할당

In [ ]:
if torch.cuda.is_available():
    k = torch.full((2, 3), 1, device=torch.device('cuda'))
    print(type(k))
    print(k.device)

### CPU로 이동

In [ ]:
if torch.cuda.is_available():
    x = x.to(torch.device('cpu'))
    print(type(x))
    print(x.device)